In [2]:
import os
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from six.moves import urllib 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# --- NEW: Import XGBoost Regressor ---
from xgboost import XGBRegressor 

ModuleNotFoundError: No module named 'xgboost'

2. Data Loading and Preparation (Re-run from Scratch)

In [2]:
# --- Data Loading Functions ---
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = "[https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz](https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz)"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    if not os.path.exists(tgz_path):
        urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

# Fetch and load data
fetch_housing_data()
housing = load_housing_data()

# --- Stratified Split ---
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6.0, np.inf], labels=[1, 2, 3, 4, 5])
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42, stratify=housing["income_cat"])
for set_ in (train_set, test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Separate features and labels
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()
housing_test = test_set.drop("median_house_value", axis=1)
housing_test_labels = test_set["median_house_value"].copy()


# --- Custom Combined Attributes Adder (from previous notebook) ---
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

# --- Full Pipeline Setup (from previous notebook) ---
housing_num = housing.drop("ocean_proximity", axis=1)
num_features = list(housing_num.columns)
cat_features = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_features),
        ("cat", OneHotEncoder(handle_unknown='ignore'), cat_features),
    ])

# Apply pipeline to training data
housing_prepared = full_pipeline.fit_transform(housing)

3. Advanced Model Benchmarking

In [ ]:
print("\n--- Advanced Model Benchmarking with Cross-Validation (RMSE) ---")

# Define model list, including XGBoost
models = [
    ("Random Forest Regressor", RandomForestRegressor(random_state=42, n_estimators=100)),
    ("XGBoost Regressor", XGBRegressor(random_state=42, n_estimators=100)) # Increased n_estimators for better comparison
]

# Run cross-validation for each model
results = []
for name, model in models:
    # Use negative MSE, then take square root and negate again for RMSE
    scores = cross_val_score(model, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10, n_jobs=-1) # Use all cores for speed
    rmse_scores = np.sqrt(-scores)
    results.append({
        'Model': name,
        'Mean RMSE': rmse_scores.mean(),
        'Std Dev': rmse_scores.std()
    })

# Display results in a clear DataFrame
benchmark_df = pd.DataFrame(results).sort_values(by='Mean RMSE')

print("\nCross-Validation Results (RMSE):\n")
print(benchmark_df.to_markdown(index=False, floatfmt=".2f"))

4. Hyperparameter Tuning on Champion Model

In [ ]:
# Assuming XGBoost is the Champion Model (adjust if Random Forest wins CV)
champion_model = XGBRegressor(random_state=42)

print("\n--- Hyperparameter Tuning on Champion Model (XGBoost) ---")
print("Tuning max_depth and learning_rate to optimize performance.")

# Define parameter grid for XGBoost tuning
param_grid = [
    {'max_depth': [3, 5], 'learning_rate': [0.05, 0.1, 0.2]}
]

# Grid Search
grid_search = GridSearchCV(champion_model, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, n_jobs=-1)

grid_search.fit(housing_prepared, housing_labels)

final_model = grid_search.best_estimator_
print(f"\nBest Model Found: {final_model}")


5. Final Evaluation and Interpretation

In [ ]:
#### 5.1 Test Set Metrics
print("\n--- Final Model Evaluation on Test Set ---")

# Prepare test data using the FIT pipeline
housing_test_prepared = full_pipeline.transform(housing_test)
final_predictions = final_model.predict(housing_test_prepared)

# Calculate RMSE
final_mse = mean_squared_error(housing_test_labels, final_predictions)
final_rmse = np.sqrt(final_mse)
print(f"Test Set RMSE: ${final_rmse:,.2f}")

# Calculate R-squared (R²)
final_r2 = r2_score(housing_test_labels, final_predictions)
print(f"Test Set R-squared (R²): {final_r2:.4f}")

In [ ]:
# --- Residual Plot ---
plt.figure(figsize=(8,6))
plt.scatter(housing_test_labels, final_predictions, alpha=0.5)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Residual Plot: Actual vs Predicted")
os.makedirs("images", exist_ok=True)
plt.savefig("images/residual_plot.png")
plt.show()


In [ ]:
print("\n--- Top 10 Feature Importances (Model Explanations) ---")

# Manually define feature names to account for all transformations
custom_features = ['rooms_per_household', 'population_per_household', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.get_feature_names_out(['ocean_proximity'])) 
final_feature_names = num_features + custom_features + cat_one_hot_attribs

feature_importances = final_model.feature_importances_
importance_df = pd.DataFrame({'Feature': final_feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False).head(10)

print("\nFeature Importance Ranking:\n")
print(importance_df.to_markdown(index=False, floatfmt=".4f"))


In [ ]:
# --- Feature Importance Plot ---
importance_df.plot(kind="barh", x="Feature", y="Importance", legend=False, figsize=(8,6))
plt.title("Top 10 Feature Importances")
plt.xlabel("Importance Score")
os.makedirs("images", exist_ok=True)
plt.savefig("images/feature_importance.png")
plt.show()


5.2 Feature Importance Ranking